랜덤포레스트로 화이트 와인을 분류하는 문제 수행해보기

In [11]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data= wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [12]:
#교차검증 수행 

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_jobs=-1, random_state=42)

In [13]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True,
                       n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


=> 훈련세트에 과대적합되었음을 알 수 있음

In [14]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


앞서 진행했던 순열중요도와 비교했을 때, 당도의 중요도가 감소했고 알콜 도수와 pH의 특성 중요도가 조금 상승하였음 

=> 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기에 하나의 특성에 과도하게 집중되지 않아서 과대적합을 줄이고 일반화 성능을 높이는데 도움 

In [15]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻었음 

OOB점수를 사용, 교차검증을 대신할 수 있음 

In [17]:
##엑스트라 트리 

from sklearn.ensemble import ExtraTreesClassifier
et= ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores= cross_validate(et, train_input, train_target, 
                       return_train_score= True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [18]:
# 특성중요도 계산 

et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [19]:
##그레디언트 부스팅을 이용해서 와인 데이터셋의 교차 검증 점수 확인

from sklearn.ensemble import GradientBoostingClassifier
gb= GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


=> 과대적합이 되지 않았음 

결정트리의 개수를 늘려도 과대적합에 매우 강함, 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음 

In [20]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores= cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 5배나 늘렸지만 과대적합을 잘 억제함 



In [21]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


랜덤포레스트보다 일부 특성인 당도에 더 집중되어있음 

In [22]:
##그레디언트 부스팅 

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb= HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면서 그레디언트 부스팅보다 조금 더 높은 성능을 제공함 

In [23]:
#특성 중요도 확인 

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
#n_repeats는 매개변수를 랜덤하게 섞을 횟수  

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준편차를 담고 있음 

평균을 출력해 보면 랜덤 포레스트와 비슷한 비율임을 알 수 있음

In [24]:
result= permutation_importance(hgb, test_input, test_target,
                               n_repeats= 10, random_state= 42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


그레디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있음을 알 수 있음

In [25]:
#테스트 세트에서의 성능을 최종적으로 확인 

hgb.score(test_input, test_target)

0.8723076923076923

약 87%의 정확도를 얻음

In [27]:
##XGBoost를 사용해 와인 데이터의 교차 검증 점수 확인

from xgboost import XGBClassifier
xgb= XGBClassifier(tree_method='hist', random_state=42)
scores= cross_validate(xgb, train_input, train_target,
                       return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [28]:
##LightGBM을 통해 와인 데이터의 교차 검증 점수 확인

from lightgbm import LGBMClassifier
lgb= LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
